In [ ]:
import torch
from pyannote.database import get_protocol, FileFinder

emb = torch.hub.load('pyannote/pyannote-audio', 'emb')
print(f'Embedding has dimension {emb.dimension:d}.')

In [ ]:
preprocessors = {'audio': FileFinder()}
protocol = get_protocol('VOXCON.SpeakerDiarization.Sample', preprocessors=preprocessors)

In [ ]:
from pyannote.audio.features.utils import get_audio_duration

train_file = next(protocol.train())
duration = get_audio_duration(train_file)

In [ ]:
from pyannote.core import SlidingWindow, Segment

sw = SlidingWindow(duration=0.025, step=0.01, start=0.0, end=duration)

for chunk in sw(Segment(3, 7.5)):
    print(tuple(chunk))

In [ ]:
def concat_segs(times, segs):
    #Concatenate continuous voiced segments
    concat_seg = []
    seg_concat = segs[0]
    for i in range(0, len(times)-1):
        if times[i][1] == times[i+1][0]:
            seg_concat = np.concatenate((seg_concat, segs[i+1]))
        else:
            concat_seg.append(seg_concat)
            seg_concat = segs[i+1]
    else:
        concat_seg.append(seg_concat)
    return concat_seg

def get_STFTs(segs):
    #Get 240ms STFT windows with 50% overlap
    sr = hp.data.sr
    STFT_frames = []
    for seg in segs:
        S = librosa.core.stft(y=seg, n_fft=hp.data.nfft,
                              win_length=int(hp.data.window * sr), hop_length=int(hp.data.hop * sr))
        S = np.abs(S)**2
        mel_basis = librosa.filters.mel(sr, n_fft=hp.data.nfft, n_mels=hp.data.nmels)
        S = np.log10(np.dot(mel_basis, S) + 1e-6)           # log mel spectrogram of utterances
        for j in range(0, S.shape[1], int(.12/hp.data.hop)):
            if j + 24 < S.shape[1]:
                STFT_frames.append(S[:,j:j+24])
            else:
                break
    return STFT_frames

def align_embeddings(embeddings):
    partitions = []
    start = 0
    end = 0
    j = 1
    for i, embedding in enumerate(embeddings):
        if (i*.12)+.24 < j*.401:
            end = end + 1
        else:
            partitions.append((start,end))
            start = end
            end = end + 1
            j += 1
    else:
        partitions.append((start,end))
    avg_embeddings = np.zeros((len(partitions),512))
    for i, partition in enumerate(partitions):
        avg_embeddings[i] = np.average(embeddings[partition[0]:partition[1]],axis=0) 
    return avg_embeddings

In [ ]:
import glob
import librosa
import numpy as np
import os
import torch

from hparam import hparam as hp
from VAD_segments import VAD_chunk

In [ ]:
audio_path = glob.glob(os.path.dirname(hp.unprocessed_data))  

total_speaker_num = len(audio_path)
train_speaker_num= (total_speaker_num//10)*9            # split total data 90% train and 10% test

audio_path

In [ ]:
next(protocol.train())['audio'].as_posix()

In [ ]:
train_sequence = []
train_cluster_id = []
label = 0
count = 0
train_saved = False
listLength = len(list(protocol.train()))
for i, file in enumerate(protocol.train()):
    filePath = file['audio'].as_posix()
    times, segs = VAD_chunk(2, filePath)
    if segs == []:
        print('No voice activity detected')
        continue
    concat_seg = concat_segs(times, segs)
#     STFT_frames = get_STFTs(concat_seg)
#     STFT_frames = np.stack(STFT_frames, axis=2)
#     STFT_frames = torch.tensor(np.transpose(STFT_frames, axes=(2,1,0)))
#     print(STFT_frames)
    embeddings = emb(file)
    aligned_embeddings = align_embeddings(embeddings.data)
    train_sequence.append(aligned_embeddings)
    for embedding in aligned_embeddings:
        train_cluster_id.append(str(label))
    count = count + 1
    if count % 100 == 0:
        print('Processed {0}/{1} files'.format(count, listLength))
    label = label + 1
    
#     if not train_saved and i > train_speaker_num:
#         train_sequence = np.concatenate(train_sequence,axis=0)
#         train_cluster_id = np.asarray(train_cluster_id)
#         np.save('train_sequence',train_sequence)
#         np.save('train_cluster_id',train_cluster_id)
#         train_saved = True
#         train_sequence = []
#         train_cluster_id = []
        
train_sequence = np.concatenate(train_sequence,axis=0)
train_cluster_id = np.asarray(train_cluster_id)
np.save('test_sequence',train_sequence)
np.save('test_cluster_id',train_cluster_id)
